<a href="https://colab.research.google.com/github/VimalMollyn/ED6001-Term-Project/blob/colab/Training/Notebooks/1.%20Test%20Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone -b colab https://github.com/VimalMollyn/ED6001-Term-Project.git


Cloning into 'ED6001-Term-Project'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 100 (delta 40), reused 47 (delta 13), pack-reused 0
Receiving objects: 100% (100/100), 9.66 MiB | 12.08 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [9]:
import torch
from torch.utils.data import Dataset
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm_notebook

In [16]:
class MRIDataset(Dataset):
    def __init__(self, path_to_data, split="train"):
        if split == "train":
            split_indices = [*range(1, 101)]
        elif split == "test":
            split_indices = [*range(101, 111)]

        print(f"Start reading {split}ing dataset...")

        # read the first set of volumes
        clean_mri_set = np.load(path_to_data / "data/1.npy")
        noisy_mri_set = np.load(path_to_data / "noisy/1.npy")

        for i in tqdm_notebook(split_indices):
            # load the current volumes
            clean_mri_temp = np.load(path_to_data / f"data/{i}.npy")
            noisy_mri_temp = np.load(path_to_data / f"noisy/{i}.npy")

            # append to the existing stack
            clean_mri_set = np.append(clean_mri_set, clean_mri_temp, axis=0)
            noised_mri_set = np.append(noisy_mri_set, noisy_mri_temp, axis=0)

        self.clean_mri_set = clean_mri_set
        self.noisy_mri_set = noisy_mri_set
        self.total = self.clean_mri_set.shape[0]
        self.current_patch = 1

        print(self.clean_mri_set.shape)
        print(f"End reading {split}ing dataset...")

    def __len__(self):
        return self.total

    def __getitem__(self, index):
        clean_img = torch.from_numpy(self.clean_mri_set[index]).float()
        noisy_img = torch.from_numpy(self.noisy_mri_set[index]).float()
        return {"clean_img": clean_img, "noisy_img": noisy_img}

In [17]:
path_to_data = Path("/content/drive/MyDrive/ED6001_MIA_Term_Project/patches")

train_dataset = MRIDataset(path_to_data=path_to_data, split="train")

Start reading training dataset...


  0%|          | 0/100 [00:00<?, ?it/s]

(79184, 1, 1, 6, 32, 32)
End reading training dataset...


In [18]:
test_dataset = MRIDataset(path_to_data=path_to_data, split="test")

Start reading testing dataset...


  0%|          | 0/10 [00:00<?, ?it/s]

(8624, 1, 1, 6, 32, 32)
End reading testing dataset...
